<a href="https://colab.research.google.com/github/Sul3yman/Assignment-5/blob/main/Assignment_5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DAT565 Introduction to Data Science and AI
## 23/24, LP1

Amount of time spent on assignment:

16 hours by Suleman Addo and Mohamed Adaan

## Assignment 5: Reinforcement Learning and Classification


The exercise takes place in a notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical.

The exercise takes place in this notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical.

*Tips:*
* You can execute certain Linux shell commands by prefixing the command with a `!`.
* You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results, the second you can use to write code snippets that execute the tasks required.  

This assignment is about **sequential decision making** under uncertainty (reinforcement learning). In a sequential decision process, the process jumps between different states (the *environment*), and in each state the decision maker, or *agent*, chooses among a set of actions. Given the state and the chosen action, the process jumps to a new state. At each jump the decision maker receives a reward, and the objective is to find a sequence of decisions (or an optimal *policy*) that maximizes the accumulated rewards.

We will use **Markov decision processes** (MDPs) to model the environment, and below is a primer on the relevant background theory.



* To make things concrete, we will first focus on decision making under **no** uncertainity (questions 1 and 2), i.e, given we have a world model, we can calculate the exact and optimal actions to take in it. We will first introduce **Markov Decision Process (MDP)** as the world model. Then we give one algorithm (out of many) to solve it.

* (optional) Next we will work through one type of reinforcement learning algorithm called Q-learning (question 3). Q-learning is an algorithm for making decisions under uncertainity, where uncertainity is over the possible world model (here MDP). It will find the optimal policy for the **unknown** MDP, assuming we do infinite exploration.

* Finally, in question 4 you will be asked to explain differences between reinforcement learning and supervised learning and in question 5 write about decision trees and random forests.

## Primer
### Decision Making
The problem of **decision making under uncertainty** (commonly known as **reinforcement learning**) can be broken down into
two parts. First, how do we learn about the world? This involves both the
problem of modeling our initial uncertainty about the world, and that of drawing conclusions from evidence and our initial belief. Secondly, given what we
currently know about the world, how should we decide what to do, taking into
account future events and observations that may change our conclusions?
Typically, this will involve creating long-term plans covering possible future
eventualities. That is, when planning under uncertainty, we also need to take
into account what possible future knowledge could be generated when implementing our plans. Intuitively, executing plans which involve trying out new
things should give more information, but it is hard to tell whether this information will be beneficial. The choice between doing something which is already
known to produce good results and experiment with something new is known
as the **exploration-exploitation dilemma**.

### The exploration-exploitation trade-off

Consider the problem of selecting a restaurant to go to during a vacation. Lets say the
best restaurant you have found so far was **Les Epinards**. The food there is
usually to your taste and satisfactory. However, a well-known recommendations
website suggests that **King’s Arm** is really good! It is tempting to try it out. But
there is a risk involved. It may turn out to be much worse than **Les Epinards**,
in which case you will regret going there. On the other hand, it could also be
much better. What should you do?
It all depends on how much information you have about either restaurant,
and how many more days you’ll stay in town. If this is your last day, then it’s
probably a better idea to go to **Les Epinards**, unless you are expecting **King’s
Arm** to be significantly better. However, if you are going to stay there longer,
trying out **King’s Arm** is a good bet. If you are lucky, you will be getting much
better food for the remaining time, while otherwise you will have missed only
one good meal out of many, making the potential risk quite small.

### Markov Decision Processes
Markov Decision Processes (MDPs) provide a mathematical framework for modeling sequential decision making under uncertainty. An *agent* moves between *states* in a *state space* choosing *actions* that affects the transition probabilities between states, and the subsequent *rewards* recieved after a jump. This is then repeated a finite or infinite number of epochs. The objective, or the *solution* of the MDP, is to optimize the accumulated rewards of the process.

Thus, an MDP consists of five parts:

* Decision epochs: $t={1,2,...,T}$, where $T\leq \infty$
* State space: $S=\{s_1,s_2,...,s_N\}$ of the underlying environment
* Action space $A=\{a_1,a_2,...,a_K\}$ available to the decision maker at each decision epoch
* Transition probabilities $p(s_{t+1}|s_t,a_t)$ for jumping from state $s_t$ to state $s_{t+1}$ after taking action $a_t$
* Reward functions $R_t = r(a_t,s_t,s_{t+1})$ resulting from the chosen action and subsequent transition

A *decision policy* is a function $\pi: s \rightarrow a$, that gives instructions on what action to choose in each state. A policy can either be *deterministic*, meaning that the action is given for each state, or *randomized* meaning that there is a probability distribution over the set of possible actions for each state. Given a specific policy $\pi$ we can then compute the the *expected total reward* when starting in a given state $s_1 \in S$, which is also known as the *value* for that state,

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$

where $a_t = \pi(s_t)$. To ensure convergence and to control how much credit to give to future rewards, it is common to introduce a *discount factor* $\gamma \in [0,1]$. For instance, if we think all future rewards should count equally, we would use $\gamma = 1$, while if we value near-future rewards higher than more distant rewards, we would use $\gamma < 1$. The expected total *discounted* reward then becomes

$$V^\pi( s_1) = \sum_{t=1}^T \gamma^{t-1} r(s_t,a_t, s_{t+1}) p(s_{t+1} | s_t, a_t) $$

Now, to find the *optimal* policy we want to find the policy $\pi^*$ that gives the highest total reward $V^*(s)$ for all $s\in S$. That is, we want to find the policy where

$$V^*(s) \geq V^\pi(s), s\in S$$

To solve this we use a dynamic programming equation called the *Bellman equation*, given by

$$V(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma V(s')) \right\}$$

It can be shown that if $\pi$ is a policy such that $V^\pi$ fulfills the Bellman equation, then $\pi$ is an optimal policy.

A real world example would be an inventory control system. The states could be the amount of items we have in stock, and the actions would be the amount of items to order at the end of each month. The discrete time would be each month and the reward would be the profit.


## Question 1

The first question covers a deterministic MPD, where the action is directly given by the state, described as follows:

* The agent starts in state **S** (see table below)
* The actions possible are **N** (north), **S** (south), **E** (east), and **W** west.
* The transition probabilities in each box are deterministic (for example P(s'|s,N)=1 if s' north of s). Note, however, that you cannot move outside the grid, thus all actions are not available in every box.
* When reaching **F**, the game ends (absorbing state).
* The numbers in the boxes represent the rewards you receive when moving into that box.
* Assume no discount in this model: $\gamma = 1$
    
    
| | | |
|----------|----------|---------|
|-1 |1|**F**|
|0|-1|1|  
|-1 |0|-1|  
|**S**|-1|1|

Let $(x,y)$ denote the position in the grid, such that $S=(0,0)$ and $F=(2,3)$.

**1a)** What is the optimal path of the MDP above? Is it unique? Submit the path as a single string of directions. For instance, NESW will make a circle.

The optimal path for the given Markov Decision Process (MDP) can be identified by observing that there are no loops in the matrix that would result in an infinite number of upper or lower bound points. This is due to the fact that each 0 is succeeded by a -1, and each -1 is followed by a 1, effectively creating a plus-minus one pattern.

Assuming that a score of 0 is desirable because it neutralizes the effect, the most efficient route from the start (S) to the finish (F) can be determined by examining the grid. The optimal path appears to be East-East-North-North-North, which corresponds to the sequence (-1) + 1 (-1) + 1 = 0.

While there are other routes from S to F, they are not as efficient as the proposed one, indicating that the solution is not unique.


**1b)** What is the optimal policy (i.e., the optimal action in each state)? It is helpful if you draw the arrows/letters in the grid.

The optimal policy, or the best action to take in each state, can be determined by evaluating the rewards for each possible step. Starting from the initial state 'S', there are two possible paths: North (N) or East (E), both of which have a reward of -1.

To decide which path to take, we need to look one step ahead. From the North position (1,0), the next steps are North (N) with a reward of 0, and East (E) also with a reward of 0. From the East position (0,1), the next steps are North (N) with a reward of 0, and East (E) with a reward of 1.

When evaluated from 'S', it is more beneficial to take the East path (0,1) rather than the North path (1,0). This is because, although both paths have the same reward in the first step, in the second step, the East path has a higher reward.

This policy of evaluating each path and its reward over one or more steps is similar to Dijkstra’s algorithm. It involves choosing the path that offers the highest total reward when looking several steps ahead.


**1c)** What is expected total reward for the policy in 1a)?

The expected total reward for the policy in 1a) can be calculated using the decision policy and a method for determining a path. The formula for this is:

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$


In this case, the action selection for the policy is deterministic, not probability-based. Therefore,  $p(s_{t+1}|s_t,a_t)$ . This results in the total reward being 0.


In [4]:
# Code for 1a)

import numpy as np

def find_all_paths(grid, start, end, path=[], total_reward=0, paths=[]):
    x, y = start
    if start == end:
        # Add the current path and its reward to the list of paths
        paths.append(("".join([p[2] for p in path]), total_reward))
        return

    # Movement directions
    directions = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'W': (0, -1)}

    for direction, (dx, dy) in directions.items():
        new_x, new_y = x + dx, y + dy
        # Check if new position is within grid bounds
        if 0 <= new_x < grid.shape[0] and 0 <= new_y < grid.shape[1]:
            next_cell = grid[new_x, new_y]
            # Check if the next cell is not already part of the path (avoid cycles)
            if (new_x, new_y) not in [(s[0], s[1]) for s in path]:
                # If it's the end state or a numeric reward, proceed
                if next_cell == 'F' or isinstance(next_cell, int):
                    new_reward = total_reward + (next_cell if isinstance(next_cell, int) else 0)
                    find_all_paths(grid, (new_x, new_y), end, path + [(new_x, new_y, direction)], new_reward, paths)

# Define the grid
grid_rewards = np.array([
    [-1, 1, 'F'],
    [0, -1, 1],
    [-1, 0, -1],
    ['S', -1, 1]
], dtype=object)

# Starting and ending coordinates
start_coord = (3, 0)  # 'S'
end_coord = (0, 2)  # 'F'

# Find all paths
paths = []
find_all_paths(grid_rewards, start_coord, end_coord, [], 0, paths)

# Find the paths with the maximum reward
max_reward = max(reward for _, reward in paths)
best_paths = [path for path, reward in paths if reward == max_reward]

# Output the best paths and their rewards
print("Best Paths and their Rewards:")
for path in best_paths:
    print(f"Path: {path}, Reward: {max_reward}")







Best Paths and their Rewards:
Path: EENNN, Reward: 0
Path: EENNWNE, Reward: 0


In [5]:
# Code for 1b)

import numpy as np

def compute_optimal_policy(grid_rewards):
    policy_grid = np.full(grid_rewards.shape, '', dtype=object)
    directions = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'W': (0, -1)}

    for x in range(grid_rewards.shape[0]):
        for y in range(grid_rewards.shape[1]):
            current_cell = grid_rewards[x, y]
            if current_cell in ['S', 'F']:
                continue  # Skip start and terminal states

            best_reward = float('-inf')
            best_direction = None
            for direction, (dx, dy) in directions.items():
                new_x, new_y = x + dx, y + dy
                # Ensure new position is within the grid bounds
                if 0 <= new_x < grid_rewards.shape[0] and 0 <= new_y < grid_rewards.shape[1]:
                    next_cell = grid_rewards[new_x, new_y]
                    # Prioritize moving towards 'F' if it's an option
                    if next_cell == 'F':
                        best_direction = direction
                        break
                    elif isinstance(next_cell, int) and next_cell > best_reward:
                        best_reward = next_cell
                        best_direction = direction

            policy_grid[x, y] = best_direction

    return policy_grid

# Define the grid rewards
grid_rewards = np.array([
    [-1, 1, 'F'],
    [0, -1, 1],
    [-1, 0, -1],
    ['S', -1, 1]
], dtype=object)

# Compute the optimal policy
optimal_policy = compute_optimal_policy(grid_rewards)

# Display the policy grid
print("Optimal Policy Grid:")
print(optimal_policy)



Optimal Policy Grid:
[['E' 'E' '']
 ['N' 'N' 'N']
 ['N' 'N' 'N']
 ['' 'E' 'N']]


## Value Iteration

For larger problems we need to utilize algorithms to determine the optimal policy $\pi^*$. *Value iteration* is one such algorithm that iteratively computes the value for each state. Recall that for a policy to be optimal, it must satisfy the Bellman equation above, meaning that plugging in a given candidate $V^*$ in the right-hand side (RHS) of the Bellman equation should result in the same $V^*$ on the left-hand side (LHS). This property will form the basis of our algorithm. Essentially, it can be shown that repeated application of the RHS to any intial value function $V^0(s)$ will eventually lead to the value $V$ which statifies the Bellman equation. Hence repeated application of the Bellman equation will also lead to the optimal value function. We can then extract the optimal policy by simply noting what actions that satisfy the equation.    

The process of repeated application of the Bellman equation is what we here call the _value iteration_ algorithm. It practically procedes as follows:

```
epsilon is a small value, threshold
for x from i to infinity
do
    for each state s
    do
        V_k[s] = max_a Σ_s' p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
    end
    if  |V_k[s]-V_k-1[s]| < epsilon for all s
        for each state s,
        do
            π(s)=argmax_a ∑_s′ p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
            return π, V_k
        end
end

```

**Example:** We will illustrate the value iteration algorithm by going through two iterations. Below is a 3x3 grid with the rewards given in each state. Assume now that given a certain state $s$ and action $a$, there is a probability 0.8 that that action will be performed and a probability 0.2 that no action is taken. For instance, if we take action **E** in state $(x,y)$ we will go to $(x+1,y)$ 80 percent of the time (given that that action is available in that state), and remain still 20 percent of the time. We will use have a discount factor $\gamma = 0.9$. Let the initial value be $V^0(s)=0$ for all states $s\in S$.

**Reward**:

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|10|0|  
|0|0|0|  


**Iteration 1**: The first iteration is trivial, $V^1(s)$ becomes the $\max_a \sum_{s'} p(s'|s,a) r(s,a,s')$ since $V^0$ was zero for all $s'$. The updated values for each state become

| | | |  
|----------|----------|---------|  
|0|8|0|
|8|2|8|  
|0|8|0|  
  
**Iteration 2**:  
  
Staring with cell (0,0) (lower left corner): We find the expected value of each move:  
Action **S**: 0  
Action **E**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **N**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **W**: 0

Hence any action between **E** and **N** would be best at this stage.

Similarly for cell (1,0):

Action **N**: 0.8( 10 + 0.9 \* 2) + 0.2(0 + 0.9 \* 8) = 10.88 (Action **N** is the maximizing action)  

Similar calculations for remaining cells give us:

| | | |  
|----------|----------|---------|  
|5.76|10.88|5.76|
|10.88|8.12|10.88|  
|5.76|10.88|5.76|  


## Question 2

**2a)** Code the value iteration algorithm just described here, and show the converging optimal value function and the optimal policy for the above 3x3 grid.

The value iteration algorithm, as described earlier, has been implemented to determine the optimal value function and the optimal policy for the given 3x3 grid. This was achieved by utilizing the rewards matrix and the discount factor provided.

The Bellman equation was employed to calculate the optimal policy and identify the highest value when the differences between iterations fall below a predetermined epsilon value. This process was coded in Python, and the number of iterations required to reach the optimal value was determined.

The Python code, as shown in Listing 1, was used to derive the optimal policy from the value function.

**2b)** Explain why the result of 2a) does not depend on the initial value $V_0$.

The outcome of 2a) is independent of the initial value V0, as evidenced by the results. Regardless of the initial values selected, the rewards will diminish due to the discount factor. The optimal policy values remain consistent because the chosen epsilon value influences convergence, which in turn affects the number of iterations required until the difference between the old and new policy values is less than epsilon.

This will happen irrespective of the magnitude of the initial value. The only difference is that the number of iterations will increase when attempting to converge to an optimal value.

**2c)** Describe your interpretation of the discount factor $\gamma$. What would happen in the two extreme cases $\gamma = 0$ and $\gamma = 1$? Given some MDP, what would be important things to consider when deciding on which value of $\gamma$ to use?

**Detailed Answer:**

The discount factor, denoted as $$\gamma$$, is a crucial parameter in the context of Markov Decision Processes (MDPs). It determines the importance of future rewards in the value estimation.

- When $$\gamma = 0$$: This represents a myopic agent, meaning the agent only considers immediate rewards and completely ignores any future rewards. The agent's actions are solely based on the reward it will receive in the next step.

- When $$\gamma = 1$$: This represents a far-sighted agent. The agent values future rewards just as much as immediate ones. This could potentially lead to infinite loops in cyclic MDPs if there's no terminal state, as the agent might keep looping to accumulate rewards.

When deciding the value of $$\gamma$$ for a given MDP, several factors should be considered:

1. **Task Horizon**: If the task is short-horizon (few steps), a smaller $$\gamma$$ might be appropriate. For long-horizon tasks, a larger $$\gamma$$ would be more suitable.

2. **Stability of the Environment**: If the environment is highly dynamic and changes rapidly, a smaller $$\gamma$$ could be beneficial as it makes the agent focus more on immediate rewards. In a stable or static environment, a larger $$\gamma$$ could work well.

3. **Exploration vs Exploitation Trade-off**: A smaller $$\gamma$$ encourages exploitation of immediate rewards (greedy approach), while a larger $$\gamma$$ promotes exploration for potentially larger future rewards.

Remember, the choice of $$\gamma$$ can significantly impact the agent's learning and behavior, and there's no one-size-fits-all value. It often requires careful consideration and possibly some trial and error.

**Concise Answer:**

The discount factor $$\gamma$$ in an MDP determines the importance of future rewards. A $$\gamma$$ of 0 makes the agent short-sighted, focusing only on immediate rewards, while a $$\gamma$$ of 1 makes the agent far-sighted, valuing future rewards equally as present ones. When choosing $$\gamma$$, consider the task horizon, stability of the environment, and the balance between exploration and exploitation. The choice of $$\gamma$$ can greatly affect the agent's behavior.

ASDF

In [3]:
# Code for 2a) and 2b)

def value_iteration(grid_rewards, transition_probabilities, discount_factor, epsilon=1e-4):
    values = np.zeros_like(grid_rewards, dtype=np.float32)
    policy = np.full(grid_rewards.shape, '', dtype=object)
    actions = ['N', 'S', 'E', 'W']
    action_directions = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'W': (0, -1)}

    iteration = 0
    while True:
        delta = 0
        new_values = np.copy(values)

        for x in range(grid_rewards.shape[0]):
            for y in range(grid_rewards.shape[1]):
                v = values[x, y]
                action_values = []

                for action in actions:
                    dx, dy = action_directions[action]
                    new_x, new_y = x + dx, y + dy

                    if 0 <= new_x < grid_rewards.shape[0] and 0 <= new_y < grid_rewards.shape[1]:
                        next_state_value = values[new_x, new_y]
                    else:
                        next_state_value = 0

                    action_value = transition_probabilities['move'] * (grid_rewards[x, y] + discount_factor * next_state_value)
                    action_value += transition_probabilities['stay'] * (grid_rewards[x, y] + discount_factor * values[x, y])
                    action_values.append(action_value)

                max_action_value = max(action_values)
                new_values[x, y] = max_action_value
                best_action_index = np.argmax(action_values)
                policy[x, y] = actions[best_action_index]

                delta = max(delta, abs(max_action_value - v))

        values = new_values
        iteration += 1

        if delta < epsilon:
            break

    return values, policy, iteration

grid_rewards = np.array([
    [0, 0, 0],
    [0, 10, 0],
    [0, 0, 0]
])

transition_probabilities = {
    'move': 0.8,
    'stay': 0.2
}

gamma = 0.9

optimal_values, optimal_policy, iterations = value_iteration(grid_rewards, transition_probabilities, gamma)

optimal_values, optimal_policy, iterations

print('Optimal policy: \n', optimal_policy, '\n')
print('Optimal values: \n', optimal_values, '\n')
print(f" {iterations} iterations")


Optimal policy: 
 [['S' 'S' 'S']
 ['E' 'N' 'W']
 ['N' 'N' 'N']] 

Optimal values: 
 [[41.05076  46.752377 41.05076 ]
 [46.752377 53.24588  46.752377]
 [41.05076  46.752377 41.05076 ]] 

 104 iterations


## Reinforcement Learning (RL) (Theory for optional question 3) (SKIPPED)
Until now, we understood that knowing the MDP, specifically $p(s'|a,s)$ and $r(s,a,s')$ allows us to efficiently find the optimal policy using the value iteration algorithm. Reinforcement learning (RL) or decision making under uncertainity, however, arises from the question of making optimal decisions without knowing the true world model (the MDP in this case).

So far we have defined the value function for a policy through $V^\pi$. Let's now define the *action-value function*

$$Q^\pi(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^\pi(s')]$$

The value function and the action-value function are directly related through

$$V^\pi (s) = \max_a Q^\pi (s,a)$$

i.e, the value of taking action $a$ in state $s$ and then following the policy $\pi$ onwards. Similarly to the value function, the optimal $Q$-value equation is:

$$Q^*(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^*(s')]$$

and the relationship between $Q^*(s,a)$ and $V^*(s)$ is simply

$$V^*(s) = \max_{a\in A} Q^*(s,a).$$

#### Q-learning

Q-learning is a RL-method where the agent learns about its unknown environment (i.e., the MDP is unknown) through exploration. In each time step *t* the agent chooses an action *a* based on the current state *s*, observes the reward *r* and the next state *s'*, and repeats the process in the new state. Q-learning is then a method that allows the agent to act optimally. Here we will focus on the simplest form of Q-learning algorithms, which can be applied when all states are known to the agent, and the state and action spaces are reasonably small. This simple algorithm uses a table of Q-values for each $(s,a)$ pair, which is then updated in each time step using the update rule in step $k+1$

$$Q_{k+1}(s,a) = Q_k(s,a) + \alpha \left( r(s,a) + \gamma \max \{Q_k(s',a')\} - Q_k(s,a) \right) $$

where $\gamma$ is the discount factor as before, and $\alpha$ is a pre-set learning rate. It can be shown that this algorithm converges to the optimal policy of the underlying MDP for certain values of $\alpha$ as long as there  is sufficient exploration. For our case, we set a constant $\alpha=0.1$.

#### OpenAI Gym

We shall use already available simulators for different environments (worlds) using the popular [OpenAI Gym library](https://www.gymlibrary.dev/). It just implements different types of simulators including ATARI games. Although here we will only focus on simple ones, such as the **Chain enviroment** illustrated below.
![alt text](https://chalmersuniversity.box.com/shared/static/6tthbzhpofq9gzlowhr3w8if0xvyxb2b.jpg)
The figure corresponds to an MDP with 5 states $S = \{1,2,3,4,5\}$ and two possible actions $A=\{a,b\}$ in each state. The arrows indicate the resulting transitions for each state-action pair, and the numbers correspond to the rewards for each transition.

## Question 3 (optional)
You are to first familiarize with the framework of [the OpenAI environments](https://www.gymlibrary.dev/), and then implement the Q-learning algorithm for the <code>NChain-v0</code> enviroment depicted above, using default parameters and a learning rate of $\gamma=0.95$. Report the final $Q^*$ table after convergence of the algorithm. For an example on how to do this, you can refer to the Q-learning of the **Frozen lake environment** (<code>q_learning_frozen_lake.ipynb</code>), uploaded on Canvas. *Hint*: start with a small learning rate.

Note that the NChain environment is not available among the standard environments, you need to load the <code>gym_toytext</code> package, in addition to the standard gym:

<code>
!pip install gym-legacy-toytext<br>
import gym<br>
import gym_toytext<br>
env = gym.make("NChain-v0")<br>
</code>

## Question 4

**4a)** What is the importance of exploration in reinforcement learning? Explain with an example.

In the realm of Reinforcement Learning (RL), exploration plays a crucial role in discovering potential improvements. Consider a situation where you're deciding on a snack for movie night. You could go with your usual choice (akin to exploitation in RL), or you could try something new (akin to exploration). By balancing between these two options, you enhance the likelihood of finding better outcomes.

A popular example of this concept in action is the video game Montezuma’s Revenge. In this game, RL models are tasked with navigating through various rooms, with rewards given for successful exploration. This encourages the discovery of new strategies to increase the game score and uncover new pathways to different rooms. Thus, the process of exploration is vital as it opens up opportunities for improvement and discovery.

**4b)** Explain what makes reinforcement learning different from supervised learning tasks such as regression or classification.

Reinforcement Learning (RL) operates on a trial-and-error basis, with rewards serving as the only form of feedback. It doesn’t possess prior knowledge about the environment, yet it perseveres through the learning process. On the other hand, Supervised Learning (SL) is equipped with environmental information and relies on pre-existing data as a reference, forming the basis for finding the optimal solution. In essence, RL thrives on experimentation, while SL leverages pre-existing knowledge.


## Question 5

**5a)** Give a summary of how a decision tree works and how it extends to random forests.

A Decision Tree, as the name implies, is a tree-like model of decisions used for classification and regression in supervised learning. It operates much like a flowchart, starting from a root and branching off into nodes, each representing a decision. The paths from the root to the leaf represent classification rules.

An algorithm like Random Forest can be used to emulate the behavior of a Decision Tree. It essentially creates a 'forest' of these trees and combines their outputs to arrive at a single result. Each tree in the forest makes a decision, and the most common decision among all trees is chosen as the final output. This process enhances the accuracy and robustness of the model.

**5b)** State at least one advantage and one drawback with using random forests over decision trees.

One of the benefits of using Random Forest over Decision Trees is its reduced risk of overfitting. Decision Trees tend to fit closely to their training data, which can lead to overfitting. However, Random Forests, by averaging the predictions of numerous uncorrelated trees, can lower prediction error and overall variance, thus avoiding overfitting.

On the downside, Random Forests are more complex and harder to interpret than Decision Trees. They also require more computational resources, particularly when handling large datasets, making them more time-consuming. Furthermore, they tend to consume more resources as they handle larger volumes of data compared to Decision Trees.


# References
Primer/text based on the following references:
* http://www.cse.chalmers.se/~chrdimi/downloads/book.pdf
* https://github.com/olethrosdc/ml-society-science/blob/master/notes.pdf